In [153]:
!pip install ucimlrepo

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install pyarrow

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import label_binarize, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.model_selection import GridSearchCV

In [110]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition = fetch_ucirepo(id=544) 
  
# data (as pandas dataframes) 
X = estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.data.features 
y = estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.data.targets 

In [111]:
X

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,Public_Transportation
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,Walking
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation


In [112]:
print(X.dtypes,'\n')
print(y.dtypes)

Gender                             object
Age                               float64
Height                            float64
Weight                            float64
family_history_with_overweight     object
FAVC                               object
FCVC                              float64
NCP                               float64
CAEC                               object
SMOKE                              object
CH2O                              float64
SCC                                object
FAF                               float64
TUE                               float64
CALC                               object
MTRANS                             object
dtype: object 

NObeyesdad    object
dtype: object


In [113]:
print(X.isna().apply(sum),'\n')
print(y.isna().apply(sum))

Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
dtype: int64 

NObeyesdad    0
dtype: int64


In [114]:
X['Gender'] = X['Gender'].map({'Female': 1, 'Male': 0})
X.rename(columns={'Gender': 'IsFemale?'}, inplace=True)

X['family_history_with_overweight'] = X['family_history_with_overweight'].map({'yes': 1, 'no': 0})

X['FAVC'] = X['FAVC'].map({'yes': 1, 'no': 0})

X['CAEC'] = X['CAEC'].map({'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3})

X['SMOKE'] = X['SMOKE'].map({'yes': 1, 'no': 0})

X['SCC'] = X['SCC'].map({'yes': 1, 'no': 0})

X['CALC'] = X['CALC'].map({'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3})

X = pd.get_dummies(X, columns=['MTRANS'])

/var/folders/q4/7q_ym6593zz06gjwqrwb7y2w0000gn/T/ipykernel_45055/604036078.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Gender'] = X['Gender'].map({'Female': 1, 'Male': 0})
/var/folders/q4/7q_ym6593zz06gjwqrwb7y2w0000gn/T/ipykernel_45055/604036078.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.rename(columns={'Gender': 'IsFemale?'}, inplace=True)
/var/folders/q4/7q_ym6593zz06gjwqrwb7y2w0000gn/T/ipykernel_45055/604036078.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [121]:
y['NObeyesdad'] = y['NObeyesdad'].map({'Normal_Weight': 0, 'Overweight_Level_I': 0, 'Overweight_Level_II': 0, 'Insufficient_Weight': 0, 'Obesity_Type_I': 1, 'Obesity_Type_II': 1, 'Obesity_Type_III': 1})
y.rename(columns={'NObeyesdad': 'IsObese?'}, inplace=True)

/var/folders/q4/7q_ym6593zz06gjwqrwb7y2w0000gn/T/ipykernel_45055/2779321678.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['NObeyesdad'] = y['NObeyesdad'].map({'Normal_Weight': 0, 'Overweight_Level_I': 0, 'Overweight_Level_II': 0, 'Insufficient_Weight': 0, 'Obesity_Type_I': 1, 'Obesity_Type_II': 1, 'Obesity_Type_III': 1})
/var/folders/q4/7q_ym6593zz06gjwqrwb7y2w0000gn/T/ipykernel_45055/2779321678.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.rename(columns={'NObeyesdad': 'IsObese?'}, inplace=True)


In [129]:
for xcolumn in X.columns:
    X[xcolumn] = X[xcolumn].astype(int)

for ycolumn in y.columns:
    y[ycolumn] = y[ycolumn].astype(int)

/var/folders/q4/7q_ym6593zz06gjwqrwb7y2w0000gn/T/ipykernel_45055/375970655.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[ycolumn] = y[ycolumn].astype(int)


In [131]:
print(X.dtypes,'\n')
print(y.dtypes)

IsFemale?                         int64
Age                               int64
Height                            int64
Weight                            int64
family_history_with_overweight    int64
FAVC                              int64
FCVC                              int64
NCP                               int64
CAEC                              int64
SMOKE                             int64
CH2O                              int64
SCC                               int64
FAF                               int64
TUE                               int64
CALC                              int64
MTRANS_Automobile                 int64
MTRANS_Bike                       int64
MTRANS_Motorbike                  int64
MTRANS_Public_Transportation      int64
MTRANS_Walking                    int64
dtype: object 

IsObese?    int64
dtype: object


In [135]:
scaler1 = StandardScaler()
scaler2 = MinMaxScaler()

for xcolumn in X.columns:
    X[xcolumn] = scaler1.fit_transform(X[[xcolumn]])
    X[xcolumn] = scaler2.fit_transform(X[[xcolumn]])

for ycolumn in y.columns:
    y[ycolumn] = scaler1.fit_transform(y[[ycolumn]])
    y[ycolumn] = scaler2.fit_transform(y[[ycolumn]])

/var/folders/q4/7q_ym6593zz06gjwqrwb7y2w0000gn/T/ipykernel_45055/1818877084.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[ycolumn] = scaler1.fit_transform(y[[ycolumn]])
/var/folders/q4/7q_ym6593zz06gjwqrwb7y2w0000gn/T/ipykernel_45055/1818877084.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[ycolumn] = scaler2.fit_transform(y[[ycolumn]])


In [157]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [159]:
knn = KNeighborsClassifier()

param_grid = {'n_neighbors': list(range(1, 25)),'weights': ['uniform', 'distance']}
metrics = ['recall','f1','accuracy']

for metric in metrics:
    grid_search = GridSearchCV(knn, param_grid, scoring=metric)
    grid_search.fit(X_train, y_train)

    print(f"best_params for {metric}: {grid_search.best_params_}")

/opt/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

best_params for recall: {'n_neighbors': 15, 'weights': 'distance'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

best_params for f1: {'n_neighbors': 7, 'weights': 'distance'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

best_params for accuracy: {'n_neighbors': 7, 'weights': 'distance'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

In [161]:
def knn_funct(X,y, test_size=0.2, n_neighbors=5, weights='uniform'):

    #tt split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

    #fit
    knn = KNeighborsClassifier()
    knn.fit(X_train, y_train)

    #predict
    y_pred = knn.predict(X_test)

    #evaluate
    rec = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    print(f"Sensitivity: {rec:.2f}")
    print(f"F1 Score: {F1:.2f}")
    print(f"Accuracy: {acc:.2f}")

In [165]:
knn_funct(X,y, n_neighbors=15, weights='distance')

Sensitivity: 0.94
F1 Score: 0.92
Accuracy: 0.93


/opt/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [167]:
knn_funct(X,y, n_neighbors=7, weights='distance')

Sensitivity: 0.95
F1 Score: 0.94
Accuracy: 0.94


/opt/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
